# MNIST

This demo shows how to create a neural network that can classify images with handwritten digits (0-9) using the [MNIST dataset](https://www.tensorflow.org/datasets/catalog/mnist).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

tf.compat.v1.disable_eager_execution() # Needed to get better graph in tensorboard

## Dataset

The dataset can be imported using TensorFlow itself.

In [ ]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
x_train.shape, y_train.shape, x_val.shape, y_val.shape

It has 60000 samples for training and 10000 samples for validation.

The inputs (`x`) are gray-scale images of 28 x 28 pixels. A pixel has a value between 0 and 255.

In [ ]:
x_train.min(), x_train.max()

The labels (`y`) are integers between 0 and 9 (inclusive).

In [ ]:
y_train.min(), y_train.max()

Let's take a look at a random example.

In [ ]:
i = np.random.choice(len(x_val))
x = x_val[i]
y_true = y_val[i]

plt.figure()
plt.imshow(x, cmap='gray')
plt.title(f'label = {str(y_true)}')
plt.show()

## Preparations

A neural network works best with values between -1 and 1. So first we will normalize the input images to have values between 0 and 1.

In [ ]:
x_train = x_train / 255
x_val = x_val / 255

x_train.min(), x_train.max()

Furthermore, classification networks usually output the probabilities that classes are present in the image. This is represented by a vector with length equal to the number of classes and a value between 0 and 1 for each class.

Out targets (i.e. ground truth) must be in this format as well, so let's convert them.

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10)

y_train.min(), y_train.max()

An example output will not look like the following.

In [ ]:
y_train[42]

## Model

Let's create the neural network itself. First we need some classes from TensorFlow.

In [ ]:
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard

The network consists of an input layer for the images (28x28). Then the image is flattened to a single vector of 784 values. Two layer will reduce that to a "feature vector" of length 64. Finally the last layer will convert it to a probability for the 10 classes.

In [ ]:
image = Input((28,28), name='image')
h = Flatten(name='flatten')(image)
h = Dense(128, activation='relu', name='dense1')(h)
h = Dense(64, activation='relu', name='dense2')(h)
labels = Dense(10, activation='softmax', name='labels')(h)
model = Model(image, labels, name='mnist')
model.summary()

We also need to specify the loss function and the optimizer. We will use the cross entropy loss function for a categorical classifier (instead of a binary classifier). The Adam optimizer performs better than the gradient descent method used earlier.

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['CategoricalAccuracy', 'Precision', 'Recall'])

## Train

Time to train the network. We give it the images as input and the labels as target and let it run for 10 epochs. The `fit` function will perform the forward propagation, compute the loss, run the backward propagation to get the gradients of the weights and then call the optimizer to change the parameters accordingly.

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, callbacks=[TensorBoard(log_dir='logs')])

## Evaluation

Let's take a random sample from the validation set and see how well it performs.

In [ ]:
def show_and_predict(i):
    x = x_val[[i]]
    y_true = np.argmax(y_val[[i]])

    plt.figure()
    plt.imshow(x[0], cmap='gray')
    plt.title(f'label = {str(y_true)}')
    plt.show()

    y_preds = model.predict_on_batch(x)[0]
    for i, p in enumerate(y_preds):
        print(f'#{i}: {p:.1f}')
    y_pred = np.argmax(y_preds)
    ok = 'OK' if y_pred == y_true else 'NOT OK'
    print(f'prediction {y_pred} => {ok}')

In [ ]:
i = np.random.choice(len(x_val))
show_and_predict(i)

Some cases are not so very clear, even for humans.

In [ ]:
show_and_predict(92)

In [ ]:
show_and_predict(4443)

In [ ]:
show_and_predict(5176)

# TensorBoard

We can use TensorBoard to visualize a lot of information about the model and the training process.

In [ ]:
!tensorboard --logdir logs --bind_all